In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2
from PIL import Image


In [2]:
def normalize_images(images):
    normalized_images = []
    
    for image_path in images:
        # Load the image using PIL
        pil_image = Image.open(image_path)
        
        # Convert PIL image to NumPy array
        np_image = np.array(pil_image)
        
        # Redimensionar a imagem para um tamanho fixo (por exemplo, 512x512)
        resized_image = cv2.resize(np_image, (512, 512))
        
        # Normalizar a imagem para ter valores entre 0 e 1
        normalized_image = resized_image.astype(np.float32) / 255.0
        
        # Adicionar a imagem normalizada à lista de imagens normalizadas
        normalized_images.append(normalized_image)
        
    return normalized_images

# Diretórios das imagens
directories = {
    "healthy_dir" : "/Users/leonardominski/Documents/Projetos Python/Pesquisa/data/Dados_PonteFerroviaria/Baseline_FrontalBogie",
    "bad95_dir" : "/Users/leonardominski/Documents/Projetos Python/Pesquisa/data/Dados_PonteFerroviaria/Validation_FrontalBogie95",
    "bad90_dir" : "/Users/leonardominski/Documents/Projetos Python/Pesquisa/data/Dados_PonteFerroviaria/Validation_FrontalBogie90",
    "bad80_dir" : "/Users/leonardominski/Documents/Projetos Python/Pesquisa/data/Dados_PonteFerroviaria/Validation_FrontalBogie80"
}

normalized_images = []
y = []


In [3]:

for directory, label in zip(directories.values(), range(len(directories))):
    images = os.listdir(directory)
    image_paths = [os.path.join(directory, image) for image in images]

    # Normalize the images by calling the "normalize_images" function
    normalized_images.extend(normalize_images(image_paths))
    
    # Create labels for the images
    y.extend([label] * len(image_paths))


In [4]:
x = np.array(normalized_images)
y = keras.utils.to_categorical(y, num_classes=len(directories))


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.2 ,random_state=42)

In [6]:
x_test.shape

(8, 512, 512, 3)

In [8]:
y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]],

In [7]:
model_cnn = keras.Sequential(
    [
        keras.Input(shape=(512, 512, 3)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.3),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(120, activation="relu"),
        layers.Dense(len(directories), activation="softmax"),
    ]
)

model_cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model_cnn.fit(x_train, y_train, batch_size=2, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5


2023-05-11 17:12:26.135599: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 